# Project Final Report: Predicting subscription status
Group number: 25

Authors: Dylan Lucas, Julie Wang, Yingnian Le

Output: html_document

## Introduction:
**provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report**

The aim of this report is to predict the usage of a Minecraft research server. 
A research group at UBC, led by Frank Wood, 

Problem: Predicting Usage of a Video Game Research Server

In particular, a research group in Computer Science at UBC, led by Frank Wood, is collecting data about how people play video games. They have set up a MineCraft server, and players' actions are recorded as they navigate through the world. But running this project is not simple: they need to target their recruitment efforts, and make sure they have enough resources (e.g., software licenses, server hardware) to handle the number of players they attract. There are three broad questions of interest.

Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

Question 2: We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.

Question 3: We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability. 

**clearly state the question you tried to answer with your project**

we are choosing to answer question 1.

**identify and fully describe the dataset that was used to answer the question**

players.csv

## Methods & Results:

**describe the methods you used to perform your analysis from beginning to end that narrates the analysis code**

your report should include code which:
- loads data 
- wrangles and cleans the data to the format necessary for the planned analysis
- performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
- creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
- performs the data analysis
- creates a visualization of the analysis 
- note: all figures should have a figure number and a legend


## Discussion:
**summarize what you found**
- discuss whether this is what you expected to find
- discuss what impact could such findings have
- discuss what future questions could this lead to


## References
You may include references if necessary, as long as they all have a consistent citation style.